In [2]:
# pip install
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install langchain-pinecone
%pip install langchain-openai python-dotenv
%pip install -U langchain langchainhub --quiet
%pip install -qU pypdf
%pip install langchain_openai

# unity-game-development
FILE_PATH = "./data/Unity Game Development in 24 Hours - Mike Geig.pdf"
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = PyPDFLoader(FILE_PATH)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone as LangchainPinecone

index_name = 'unity-game-development'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# Create database from existing index
database = LangchainPinecone.from_existing_index(index_name=index_name, embedding=embedding)
retrieved_pdf = database.similarity_search(query, k=3)

from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
query = 'How do I set up character animations in Unity?'

prompt = f"""[Identity]
- You are an expert in Unity game development. Please use the information from the [Context] section to provide accurate and detailed answers related to Unity game development.

[Context]
{retrieved_pdf}

Question : {query}
"""

ai_message = llm.invoke(prompt)
ai_message.content


ValueError: File path ./data/Unity Game Development in 24 Hours - Mike Geig.pdf is not a valid file or url